In [20]:
from selenium import webdriver
import os
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
from datetime import datetime
import db_multiproc_calc_neutral2 as dmc
from random import *

rf_rate = 0.0475
k_do_download = True

cwd = os.getcwd()

executable_path = os.path.join(cwd,'chromedriver.exe')
display(executable_path)

date_str = datetime.now().strftime("%Y-%m-%d")
date_str = '2023-03-24'
file_date_str = date_str

download_path = os.path.join(cwd, 'local_download')
file_date_str

'C:\\Users\\mcbri\\PycharmProjects\\futureDataCapture\\chromedriver.exe'

'2023-03-24'

In [21]:
def bar_chart_get_driver(executable_path, download_path):
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_experimental_option("prefs", {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })
    #chromeOptions.add_argument("--window-size=50000,50000")
    driver = webdriver.Chrome(executable_path=executable_path, options=chromeOptions)
    return driver

def bar_chart_login(driver):
    driver.get("https://www.barchart.com/login")
    driver.find_element(By.NAME, "email").click()
    driver.find_element(By.NAME, "email").send_keys("mcbride1689@gmail.com")
    driver.find_element(By.NAME, "password").click()
    driver.find_element(By.NAME, "password").send_keys("d5nJSz57A8C5")
    driver.find_element_by_class_name('login-button').click()
    return



In [22]:
#############################
# get the Futures contracts and months we are needing and build a list of all the distinct contracts

df_future_month_def = pd.read_csv('futures_months.csv')

target_years = [23]

all_syms = []
for idx, row in df_future_month_def.iterrows():
    for y in target_years:
        for m in row['Month']:
            all_syms.append(row['symbol_root']+m+f'{y:.0f}')

In [23]:
if k_do_download:
    driver = bar_chart_get_driver(executable_path, download_path)
    bar_chart_login(driver)

In [24]:
def get_futures_contract_specs(sym):
    contract_root_url = f"https://www.barchart.com/futures/quotes/{sym}/overview"
    option_greek_url = f'https://www.barchart.com/futures/quotes/{sym}/volatility-greeks?moneyness=allRows'

    #-------------
    # open the contract root page
    driver.get(contract_root_url)

    # get the first notice and expiration date
    time.sleep(random()*3)
    # get the expiration of the contract from the option price page
    temp_name = "#main-content-column > div > div.barchart-content-block.commodity-profile > div.block-content > div > div:nth-child(2) > div:nth-child(4) > div.small-7.column.text-right > span > span"
    first_notice = driver.find_element(By.CSS_SELECTOR, temp_name).text
    first_notice = first_notice[0:8]

    temp_name = "#main-content-column > div > div.barchart-content-block.commodity-profile > div.block-content > div > div:nth-child(2) > div:nth-child(5) > div.small-7.column.text-right > span > span"
    futures_expiration = driver.find_element(By.CSS_SELECTOR, temp_name).text
    # make sure we only get the date portion
    futures_expiration = futures_expiration[0:8]

    #-------------
    # open the greeks page to get options contract details. We open each contract page and get the top 4 months
    driver.get(option_greek_url)
    time.sleep(random()+1)
    try:
        # Get the list of options in the dropdown
        driver.find_element(By.ID, "bc-options-toolbar__dropdown-month").click()
        dropdown = driver.find_element(By.ID, "bc-options-toolbar__dropdown-month")
        options = dropdown.text
        options = options.split('\n')
        options = [x.lstrip() for x in options]
        mon_expr=[]
        option_expiration=[]
        # iterate through the options and get the expiration
        for i in [0, 1, 2]:
            time.sleep(random()*3)
            try:
                driver.find_element(By.ID, "bc-options-toolbar__dropdown-month").click()
                dropdown = driver.find_element(By.ID, "bc-options-toolbar__dropdown-month")
                dropdown.find_element(By.XPATH, f"//option[. = '{options[i]}']").click()

                # Get the contract associated with the month
                temp_name = "#main-content-column > div > div.page-title.symbol-header-info.ng-scope > div.symbol-name > div > span:nth-child(2)"
                # note we remove ( and ) by removing the first and last character
                mon_expr.append(driver.find_element(By.CSS_SELECTOR, temp_name).text[1:-1])

                temp_name = "#main-content-column > div > div:nth-child(4) > div > div:nth-child(1) > div > strong:nth-child(2)"
                option_expiration.append(driver.find_element(By.CSS_SELECTOR, temp_name).text)
            except:
                print(options[i] + " not found")
    except:
        mon_expr = ['']
        option_expiration = ['']

    ret_data = [first_notice, futures_expiration, mon_expr,option_expiration]
    return ret_data

In [25]:
contract_specs = []
for sym in all_syms:
    print(sym)
    time.sleep(random()*2)
    contract_specs.append(get_futures_contract_specs(sym))

ZFH23
ZFM23
ZFU23
ZFZ23
ZTH23
ZTM23
ZTU23
ZTZ23
SQF23
SQG23
SQH23
SQJ23
SQK23
SQM23
SQN23
SQQ23
SQU23
SQV23


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#main-content-column > div > div.barchart-content-block.commodity-profile > div.block-content > div > div:nth-child(2) > div:nth-child(4) > div.small-7.column.text-right > span > span"}
  (Session info: chrome=111.0.5563.111)


In [26]:
df_contracts = pd.DataFrame([all_syms, contract_specs]).T

In [27]:
df_contracts.to_csv('future_contract_spec.2.csv')

In [28]:
driver.quit()